In [ ]:
import gspread
from google.oauth2.service_account import Credentials

# Load credentials from JSON file - MAKE SURE TO MAKE ONE FIRST, ENABLE GOOGLE DRIVE AND GOOGLE SHEETS API (ask chatgpt)
# Replace the form-response-resume-extracter thing with your credentials json file to use it
SCOPES = ["https://www.googleapis.com/auth/spreadsheets", "https://www.googleapis.com/auth/drive"]
creds = Credentials.from_service_account_file("form-response-resume-extracter.json", scopes=SCOPES)

# Authorize gspread
client = gspread.authorize(creds)

# Open the Google Sheet
SPREADSHEET_ID = "13Yz7_yaocfgUJ3LrSatAcGAUAXzmHT_-jP322cR3oHY"  # Replace with actual ID from the sheet URL
sheet = client.open_by_key(SPREADSHEET_ID).sheet1

# Get all data from the applications sheet
data = sheet.get_all_records()

In [26]:
import pandas as pd
import re

def snake_case(s):
    s = re.sub(r"[-.\s]", '_', str(s))
    return '_'.join(re.findall(r"[A-Za-z0-9]+", s)).lower()

admits_df = pd.DataFrame(data)
admits_df['Snake Case Name'] = [snake_case(name) for name in admits_df['Name']]

In [ ]:
admits_df["Resume Name"] = "resume_" + admits_df['Snake Case Name'] + "_" + admits_df.index.astype(str) + ".pdf"
resume_key = "Please upload your resume again!"
going = admits_df[admits_df['Will you be competing at the 2025 SCPT? Please note that this is binding!'] == 'Yes']
pdf_links = going.set_index('Resume Name')[resume_key].to_dict()

In [ ]:
import os
import requests
import re

def extract_file_id(url):
    match = re.search(r"(?:id=|/d/|uc\?id=)([a-zA-Z0-9_-]+)", url)
    return match.group(1) if match else None

# Create the Drive API service
from googleapiclient.discovery import build
drive_service = build("drive", "v3", credentials=creds)

# Folder to save PDFs
output_folder = "./2025_scpt_resumes"
os.makedirs(output_folder, exist_ok=True)

def download_pdf(file_id, save_path):
    """Download a PDF from Google Drive using file ID."""
    try:
        request = drive_service.files().get_media(fileId=file_id)
        with open(save_path, "wb") as f:
            f.write(request.execute())
        # print(f"Downloaded: {save_path}")
    except Exception as e:
        print(f"Failed to download {file_id}: {e}")

for file_name, url in pdf_links.items():
    file_id = extract_file_id(url)
    if file_id:
        file_path = os.path.join(output_folder, file_name)
        try:
            download_pdf(file_id, file_path)
            print(f"Downloaded: {file_path}")
        except Exception as e:
            print(f"Failed to download {file_id}: {e}")
    else:
        print(f"Invalid URL: {url}")